In [ ]:
# Run the line below to bring zfel to the system
#! git clone https://github.com/slaclab/zfel.git

In [ ]:
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.style.use(r'PaperDoubleFig.mplstyle')

import os, sys
sys.path.insert(0, os.path.abspath('zfel'))
from zfel.old_scripts import mingxie

In [ ]:
?mingxie

In [ ]:
XLAMD = 1.86e-2  # undulator period
XLAMDS = 0.3e-10  # desired x-ray wavelength
CURPEAK = 3e3  # peak current
GAMMA0 = 12e9/0.511e6  # energy in mc2
energy = 0.511e6*GAMMA0  # beam energy in eV
DELGAM = 1.5e-4
sigma_e = DELGAM*energy  # energy spread in eV
AWO = np.sqrt(2*GAMMA0**2*XLAMDS/XLAMD-1)  # we use resonant condition formula here
K = np.sqrt(2)*AWO
EMITX = 0.2e-6  # normalized emittance in x
beta = 15  # beta function in meters
RXBEAM = np.sqrt(beta*EMITX/GAMMA0)  # the corresponding beam size

In [ ]:
# Some parameters
params = {    
    'sigma_x':RXBEAM,
    'und_lambda':XLAMD,
    'und_k':K,
    'current':CURPEAK,
    'gamma':GAMMA0,
    'norm_emit':EMITX,
    'sigma_E':sigma_e}


mingxie(**params)

# Beta scanning

In [ ]:
n = 200
params2 = {}
# Duplicate all values
for k,v in params.items():
    params2[k] = np.array(n*[v])

# Vary beta 
beta = np.linspace(5, 30, n)
params2['sigma_x'] = np.sqrt(beta*EMITX/GAMMA0)

# Calculate
result = mingxie(**params2)   

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel(r'Average $\beta$ (m)')
ax.set_ylabel('Number of gain length to saturation')
ax.plot(beta, result['saturation_length']/result['gain_length'])
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
plt.title(r"Ming Xie predictions for different focusing")
ax1.plot(beta, result['saturation_length'])
ax1.set_xlabel(r'Average $\beta$ (m)')
ax1.set_ylabel('Saturation length (m)')

ax2 = ax1.twinx()
ax2.plot(beta, 1e-9*result['saturation_power'], 'r')
ax2.set_ylabel(r"Saturation Power, GW", color='r')
ax2.tick_params('y', colors='r')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(beta, result['saturation_power']/(1.6*result['pierce_parameter']*CURPEAK*energy))
ax.set_xlabel(r'Average $\beta$ (m)')
ax.set_ylabel(r'FEL efficiency, $\frac{P_{sat}}{1.6\rho I_0 E }$')
plt.show()

# Emittance and Energy spread

In [ ]:
XLAMD = 1.86e-2  # undulator period
XLAMDS = 0.3e-10  # desired x-ray wavelength
CURPEAK = 3e3  # peak current
GAMMA0 = 12e9/0.511e6  # energy in mc2
energy = 0.511e6*GAMMA0  # beam energy in eV
DELGAM = 1.5e-4
sigma_e = DELGAM*energy  # energy spread in eV
AWO = np.sqrt(2*GAMMA0**2*XLAMDS/XLAMD-1)  # we use resonant condition formula here
K = np.sqrt(2)*AWO
EMITX = 0.2e-6  # normalized emittance in x
beta = 15  # beta function in meters
RXBEAM = np.sqrt(beta*EMITX/GAMMA0)  # the corresponding beam size

In [ ]:
n = 50
params2 = {}
# Duplicate all values
for k, v in params.items():
    params2[k] = np.array(n*n*[v])
sigma_e = np.linspace(0.1e-4, 3e-4)
EMITX = np.linspace(0.1e-6, 0.5e-6)
X, Y = np.meshgrid(EMITX, sigma_e)
params2['norm_emit'], params2['sigma_E'] = X.flatten(), Y.flatten()*energy
params2['sigma_x'] = np.sqrt(beta*params2['norm_emit']/GAMMA0)
result = mingxie(**params2) 

In [ ]:
fig, ax = plt.subplots()
c = ax.contour(1e6*X, 1e4*Y, 1e-9*result['saturation_power'].reshape(50,50), 15)
#plt.colorbar()
ax.clabel(c, inline=1, fontsize=14, fmt='%1.1f')
ax.set_title('Saturation Power (GW)')
ax.set_xlabel('Normalized Emittance (mm-mrad)')
ax.set_ylabel(r'Relative energy spread ($\times 10^{-4}$)')
plt.show()

In [ ]:
plt.contourf(1e6*X, 1e4*Y, 1e-9*result['saturation_power'].reshape(50,50), 15)
plt.colorbar()
plt.title('Saturation Power (GW)')
plt.xlabel('Normalized Emittance (mm-mrad)')
plt.ylabel(r'Relative energy spread ($\times 10^{-4}$)')
plt.show()